# 1. Install dependencies and setup

In [1]:
!pip install tflearn
!conda install -y gdown


import math
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import keras
from tqdm import tqdm
import csv  
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from random import shuffle
# from google.colab import drive
import tensorflow as tf
import tflearn
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.models import Model
from keras import activations
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    MaxPooling2D, GlobalAveragePooling2D,\
    Flatten, Rescaling, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.models import save_model
from keras.utils import data_utils
from keras.layers import VersionAwareLayers
from tflearn.layers.core import input_data
from tensorflow.keras import layers
from keras.engine import training

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 545.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=fd0f5881d0de8677b1608d0b41c003bd0e4520c72b4f2ff9a091638dbbe7b2a9
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0  


# 2. Loading the data via `gdrive`

In [2]:
# drive.mount('/content/drive')
!gdown --id 1s07aL-7nvhO8ESJy_uTCZJMF5lw5LAGK
!unzip "/kaggle/working/NN Dataset.zip"

train_dir ='/kaggle/working/Train'
test_dir ='/kaggle/working/Test'

train_filenames = os.listdir(train_dir)
test_filenames = os.listdir(test_dir)

labels = sorted(set([fname.split('_')[0] for fname in train_filenames]))

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1s07aL-7nvhO8ESJy_uTCZJMF5lw5LAGK
To: /kaggle/working/NN Dataset.zip
100%|█████████████████████████████████████████| 274M/274M [00:01<00:00, 243MB/s]
Archive:  /kaggle/working/NN Dataset.zip
  inflating: Test/0.jpg              
  inflating: Test/1.jpg              
  inflating: Test/10.jpg             
  inflating: Test/100.jpg            
  inflating: Test/101.jpg            
  inflating: Test/102.jpg            
  inflating: Test/103.jpg            
  inflating: Test/104.jpg            
  inflating: Test/105.jpg            
  inflating: Test/106.jpg            
  inflating: Test/107.jpg            
  inflating: Test/108.jpg            
  inflating: Test/109.jpg            
  inflating: Test/11.jpg             
 

# 3. Preprocessing

1.   Converting all images to `jpg`
2.   Resizing all images to `(255,255,3)` 
3.   Placing each class in a subfolder
4.   Augmenting dataset




### 3.1 Placing each class in a subfolder 

In [3]:
# Create a sub-folder for each label
labels = sorted(set([fname.split('_')[0] for fname in train_filenames]))

for label in labels:
  os.mkdir(f'{train_dir}/{label}')


# Move each image to it's corresponding sub-folder
for file in train_filenames:
  label = file.split('_')[0]
  shutil.move(f'{train_dir}/{file}', f'{train_dir}/{label}/{file}')

# 4. Preparing train/ validation/ test data to feed the model



Converting train/ val to `BatchDataset`



In [4]:
IMG_SIZE = 250

# Get the data
trainIMG = ImageDataGenerator(validation_split=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            rotation_range = 40,
                            vertical_flip=True)
trainingDataset = trainIMG.flow_from_directory(directory="/kaggle/working/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='training')


validationnDataset = trainIMG.flow_from_directory(directory="/kaggle/working/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='validation')


Found 1346 images belonging to 6 classes.
Found 335 images belonging to 6 classes.


# 5. Network Architecture

In [5]:
channel_axis = -1
img_input = layers.Input(shape=[IMG_SIZE, IMG_SIZE, 3])
 # Entry block 
modelLayers = layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name="block1_conv1")(img_input)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block1_conv1_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block1_conv1_act")(modelLayers)
modelLayers = layers.Conv2D(64, (3, 3), use_bias=False, name="block1_conv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block1_conv2_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block1_conv2_act")(modelLayers)

residual = layers.Conv2D(128, (1, 1), strides=(2, 2), padding="same", use_bias=False)(modelLayers)
residual = layers.BatchNormalization(axis=channel_axis)(residual)

modelLayers = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv1")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv1_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block2_sepconv2_act")(modelLayers)
modelLayers = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv2_bn")(modelLayers)

modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block2_pool")(modelLayers)
modelLayers = layers.add([modelLayers, residual])

residual = layers.Conv2D(256, (1, 1), strides=(2, 2), padding="same", use_bias=False)(modelLayers)
residual = layers.BatchNormalization(axis=channel_axis)(residual)

modelLayers = layers.Activation("relu", name="block3_sepconv1_act")(modelLayers)
modelLayers = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv1")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv1_bn")( modelLayers)
modelLayers = layers.Activation("relu", name="block3_sepconv2_act")(modelLayers)
modelLayers = layers.SeparableConv2D( 256, (3, 3), padding="same", use_bias=False, name="block3_sepconv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv2_bn")(modelLayers)

modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block3_pool")(modelLayers)
modelLayers = layers.add([modelLayers, residual])

residual = layers.Conv2D(728, (1, 1), strides=(2, 2), padding="same", use_bias=False)(modelLayers)
residual = layers.BatchNormalization(axis=channel_axis)(residual)

modelLayers = layers.Activation("relu", name="block4_sepconv1_act")(modelLayers)
modelLayers = layers.SeparableConv2D( 728, (3, 3), padding="same", use_bias=False, name="block4_sepconv1")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv1_bn")( modelLayers)
modelLayers = layers.Activation("relu", name="block4_sepconv2_act")(modelLayers)
modelLayers = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv2_bn")(modelLayers)

modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block4_pool" )(modelLayers)
modelLayers = layers.add([modelLayers, residual])

 # Middle block 

for i in range(8):
  residual = modelLayers
  prefix = "block" + str(i + 5)

  modelLayers = layers.Activation("relu", name=prefix + "_sepconv1_act")(modelLayers)
  modelLayers = layers.SeparableConv2D(728,(3, 3), padding="same", use_bias=False, name=prefix + "_sepconv1", )(modelLayers)
  modelLayers = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv1_bn")(modelLayers)
  modelLayers = layers.Activation("relu", name=prefix + "_sepconv2_act")(modelLayers)
  modelLayers = layers.SeparableConv2D( 728,(3, 3),padding="same", use_bias=False,name=prefix + "_sepconv2",)(modelLayers)
  modelLayers = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv2_bn")(modelLayers)
  modelLayers = layers.Activation("relu", name=prefix + "_sepconv3_act")(modelLayers)
  modelLayers = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv3",)(modelLayers)
  modelLayers = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv3_bn")(modelLayers)

  modelLayers = layers.add([modelLayers, residual])
    
# Exit block 

residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding="same", use_bias=False)(modelLayers)
residual = layers.BatchNormalization(axis=channel_axis)(residual)

modelLayers = layers.Activation("relu", name="block13_sepconv1_act")(modelLayers)
modelLayers = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block13_sepconv1")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv1_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block13_sepconv2_act")(modelLayers)
modelLayers = layers.SeparableConv2D(1024, (3, 3), padding="same", use_bias=False, name="block13_sepconv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv2_bn")(modelLayers)

modelLayers = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block13_pool")(modelLayers)
modelLayers = layers.add([modelLayers, residual])

modelLayers = layers.SeparableConv2D(1536, (3, 3), padding="same", use_bias=False, name="block14_sepconv1")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv1_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block14_sepconv1_act")(modelLayers)

modelLayers = layers.SeparableConv2D(2048, (3, 3), padding="same", use_bias=False, name="block14_sepconv2")(modelLayers)
modelLayers = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv2_bn")(modelLayers)
modelLayers = layers.Activation("relu", name="block14_sepconv2_act")(modelLayers)

modelWithoutweights = training.Model(img_input, modelLayers, name="Xception")

TF_WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"
)

weights_path = data_utils.get_file("xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
                TF_WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="b0042744bf5b25fce3cb969f33bebb97",
                )
modelWithoutweights.load_weights(weights_path)

modelWithweights = Sequential()

modelWithweights.add(modelWithoutweights)
modelWithweights.add(GlobalAveragePooling2D())
modelWithweights.add(Flatten())
modelWithweights.add(Dense(4096,activation='relu'))
modelWithweights.add(Dropout(0.5))
modelWithweights.add(Dense(6,activation='softmax'))


print("done")

83697664/83683744 [==============================] - 3s 0us/step


2022-12-25 21:19:01.312134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:19:01.462355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:19:01.463236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 21:19:01.464943: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

done


In [6]:
modelWithweights.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              8392704   
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 24582     
Total params: 29,278,766
Trainable params: 29,224,238
Non-trainable params: 54,528
_______________________________________

# 6. Model training

In [7]:
modelWithweights.compile(Adam(learning_rate=0.0001),
              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


checkpoint = ModelCheckpoint("Xcepiton.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
print("done")

done


In [8]:
history = modelWithweights.fit(trainingDataset, epochs=10, validation_data=validationnDataset, callbacks=[checkpoint])

Epoch 1/10


2022-12-25 21:19:20.489067: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 1.0283 - acc: 0.6523

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.81493, saving model to Xcepiton.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


43/43 [==============================] - 56s 867ms/step - batch: 21.0000 - size: 31.3023 - loss: 1.0281 - acc: 0.6523 - val_loss: 0.5181 - val_acc: 0.8149
Epoch 2/10
43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.2890 - acc: 0.9079
Epoch 00002: val_acc improved from 0.81493 to 0.94925, saving model to Xcepiton.h5
43/43 [==============================] - 35s 824ms/step - batch: 21.0000 - size: 31.3023 - loss: 0.2889 - acc: 0.9079 - val_loss: 0.1812 - val_acc: 0.9493
Epoch 3/10
43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.2145 - acc: 0.9354
Epoch 00003: val_acc improved from 0.94925 to 0.95522, saving model to Xcepiton.h5
43/43 [==============================] - 36s 850ms/step - batch: 21.0000 - size: 31.3023 - loss: 0.2142 - acc: 0.9354 - val_loss: 0.1189 - val_acc: 0.9552
Epoch 4/10
43/43 [==============================] - ETA: 0s - batch: 21.0000 - size: 31.3023 - loss: 0.1319 - acc: 0.9643
Epo

# 7. Model Prediction

In [12]:
import os
os.chdir(r'/kaggle/working/')
from IPython.display import FileLink
FileLink(r'/kaggle/working/Xcepiton.h5')


/kaggle/working/Xcepiton.h5

In [10]:
from keras.models import load_model
loadedModel = load_model('/kaggle/working/Xcepiton.h5')
print("Done")

Done


In [11]:
with open('Xception.csv', 'w', encoding='UTF8',newline='') as CsvWriter:
  header = ["image_name","label"]
  writer = csv.writer(CsvWriter)
  writer.writerow(header)
  for img in tqdm(os.listdir(test_dir)):
    path = os.path.join(test_dir, img)
    img_arr = tf.keras.utils.load_img(path,target_size=(IMG_SIZE,IMG_SIZE))
    img_arr = np.asarray(img_arr)
    img_arr = np.expand_dims(img_arr, axis=0)
    prediction = loadedModel.predict(img_arr)
    data = [img,np.argmax(prediction)]
    writer.writerow(data)

100%|██████████| 688/688 [00:17<00:00, 38.76it/s]
